In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
data = pd.read_csv("csvFiles/DATASET02_size_222.csv")
data.drop(columns = ['Unnamed: 0'], inplace = True)
X = data['issue']
y = data['urgency']

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 600)
X_vectorized = vectorizer.fit_transform(X)

In [5]:
X_dense = X_vectorized.toarray()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dense, y, test_size=0.2, random_state=42)

In [13]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate = 0.04), loss='mean_squared_error')
model.fit(X_train, y_train, epochs=35, batch_size=16, verbose=0)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

2/2 [==============================] - 0s 862us/step
0.8448486299553101
0.007863270640562223
0.1336595662382269


In [14]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 128)               76928     
                                                                 
 dense_25 (Dense)            (None, 64)                8256      
                                                                 
 dense_26 (Dense)            (None, 64)                4160      
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 89409 (349.25 KB)
Trainable params: 89409 (349.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
import numpy as np
complaint = ["Extremely urgent to address the unauthorised transaction of 100 dollars from my bank account yesterday. Did not receive any refund"]

#DO NOT USE fit_transform!!!!!!You need to transform the new data on previously fit vectorizer

complaint_vec = vectorizer.transform(complaint)
complaint_vec = complaint_vec.toarray()
urgencyval = model.predict(complaint_vec)
print(urgencyval)

1/1 [==============================] - 0s 14ms/step
[[0.75239134]]


In [11]:
#different urgency complaints
complaints = [
    ["I have noticed an unauthorized transaction on my account dated May 10th for $50. This transaction was not made by me, and I am very concerned about the security of my account. I need this resolved immediately as it is affecting my balance and causing me significant stress. Additionally, I would like to know what measures are being taken to prevent such incidents in the future."],
    ["My credit card payment due date was suddenly changed without any notification. This caused me to miss the payment, which I believe is unfair and needs to be rectified. I have always been prompt with my payments, and this sudden change has disrupted my financial planning. Please address this issue regarding the expiry of my card, as it was not my fault."],
    ["I have been charged twice for the same purchase at your store on April 25th. This has resulted in an unnecessary deduction from my account, and I am now short on funds for other expenses. Please refund the duplicate charge as soon as possible and confirm that this will not happen again. I also suggest reviewing your payment processing system to avoid such errors."],
    ["I received my monthly statement, and there is a misprint in my name. It is spelled incorrectly and should be corrected in your records. This kind of error can lead to issues with identity verification and other banking services. Please update your records and send me a corrected statement. I would appreciate confirmation once this is done."],
    ["There is a typo in my passbook for the entry on March 3rd. It shows the wrong transaction description, which is a minor issue but needs correction for accurate record-keeping. Maintaining correct records is important for my financial tracking. Please update the passbook entry and send me a corrected version. This will help me keep my financial documents in order."]
]

#all 5 are complaints related to unauth transations
# complaints = [
#     ["I noticed an unauthorized transaction on my account yesterday for $200. I didn't make this purchase and need it reversed immediately. This is causing me a lot of anxiety, and I hope it gets resolved soon."],
#     ["There is a suspicious charge of $150 on my account from a vendor I've never heard of. I demand an immediate investigation into this matter. How can such a transaction even go through without my authorization?"],
#     ["I just saw an unfamiliar transaction on my bank statement for $75. I certainly did not authorize this. Can someone please look into this and get back to me with the next steps?"],
#     ["I am writing to report a fraudulent charge of $500 on my credit card. This transaction is not mine, and I require urgent assistance to have it removed. Additionally, please ensure my account's security to prevent future occurrences."],
#     ["There has been an unauthorized transaction on my account for $300. I am very concerned about how this happened and insist on a thorough investigation. Please provide me with updates on the progress and steps being taken to secure my account."]
# ]


sortedComplaints = []
for complaint in complaints:
    complaint_vec = vectorizer.transform(complaint)
    complaint_vec = complaint_vec.toarray()
    urgencyval = model.predict(complaint_vec)
    sortedComplaints.append([complaint, urgencyval])
print(sortedComplaints)

1/1 [==============================] - 0s 7ms/step
[[['I have noticed an unauthorized transaction on my account dated May 10th for $50. This transaction was not made by me, and I am very concerned about the security of my account. I need this resolved immediately as it is affecting my balance and causing me significant stress. Additionally, I would like to know what measures are being taken to prevent such incidents in the future.'], array([[0.8539581]], dtype=float32)], [['My credit card payment due date was suddenly changed without any notification. This caused me to miss the payment, which I believe is unfair and needs to be rectified. I have always been prompt with my payments, and this sudden change has disrupted my financial planning. Please address this issue regarding the expiry of my card, as it was not my fault.'], array([[0.8328156]], dtype=float32)], [['I have been charged twice for the same purchase at your store on April 25th. This has resulted in an unnecessary deduction